In [1]:
import torch
from torch import nn

import numpy

In [ ]:
mse = nn.MSELoss()

In [8]:
a = torch.rand(300, 128, 2400)
a.size()

torch.Size([300, 128, 2400])

In [9]:
b = a.transpose(0, 1)
b.size()

torch.Size([128, 300, 2400])

In [10]:
c = b[:, 1:].reshape(-1, b.size(-1))

In [11]:
c.size()

torch.Size([38272, 2400])